In [1]:
import mne
import numpy as np
import pandas as pd
from sklearn import preprocessing 
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

import os

mne.set_log_level('WARNING')

event_id_all = {'0Suff w/ Lat.': 1,
                '0Suff w/o Lat.': 2,
                '0Suff NW': 4,

                '1Suff w/ Lat.': 11,
                '1Suff w/o Lat.': 12,
                '1Suff PseudoStemNW': 14,
                '1Suff RealStemNW': 15,

                '2Suff w/ Lat.': 21,
                '2Suff w/o Lat.': 22,
                '2Suff Composite': 23,
                '2Suff PseudoStemNW': 24,
                '2Suff RealStemNW': 25
                }

def get_condition(id):
    for event in event_id_all:
        if event_id_all[event] == id:
            return event

import mne
from glob import glob
from tqdm import tqdm

epoch_files = glob('/scratch/alr664/multiple_affix/meg/*/*epo.fif')

ImportError: libgomp-a34b3233.so.1.0.0: cannot open shared object file: No such file or directory
___________________________________________________________________________
Contents of /home/rss9311/.local/lib/python3.12/site-packages/sklearn/__check_build:
__init__.py               _check_build.cpython-312-x86_64-linux-gnu.so__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.

In [20]:
!pwd

/scratch/rss9311/neurips2024/neurips-analyses


In [21]:
root = '/scratch/alr664/multiple_affix'
meg = root + '/meg'
logs = root + '/logs'

full_dataset = ["A0394", "A0421", "A0446", "A0451", "A0468", "A0484", "A0495", "A0502", "A0503", "A0508", 
                "A0509", "A0512", "A0513", "A0514", "A0516", "A0517", "A0518", "A0519", "A0520", "A0521", 
                "A0522", "A0523", "A0524", "A0525"]

subjects = [subj for subj in os.listdir(meg) if not subj.startswith('.')]
subjects
len(subjects)

epoch_files = []
X = []
y = []

for subject in full_dataset:
    subj_epoch_path = meg + '/' + subject + '/' + subject + '_rejection-epo.fif'
    print(subj_epoch_path)
    epoch_files.append(subj_epoch_path)

#print(epoch_files)

/scratch/alr664/multiple_affix/meg/A0394/A0394_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0421/A0421_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0446/A0446_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0451/A0451_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0468/A0468_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0484/A0484_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0495/A0495_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0502/A0502_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0503/A0503_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0508/A0508_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0509/A0509_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0512/A0512_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0513/A0513_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0514/A0514_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0516/A0516_rejection-epo.fif
/scratch/alr664/multiple_

In [22]:
## 3-class classification code

import mne
from glob import glob
from tqdm import tqdm

X = []
y = []
meg_full_data = [] 
labels_full_data = []
for epoch_file in tqdm(epoch_files):
    print(epoch_file)
    epochs = mne.read_epochs(epoch_file)
    epochs = epochs.resample(125)
    epochs = epochs.crop(tmin=0. , tmax= 0.6)
    #print(epochs.times)
    # combine for 3-class classification
    #epochs = mne.epochs.combine_event_ids(epochs, ['0Suff NW', '0Suff w/o Lat.', '0Suff w/ Lat'],  {'0Suff': 100},  True)
    #epochs = mne.epochs.combine_event_ids(epochs, ['1Suff PseudoStemNW', '1Suff RealStemNW', '1Suff w/ Lat.', '1Suff w/o Lat.'],  {'1Suff': 101},  True)
    #epochs = mne.epochs.combine_event_ids(epochs, ['2Suff RealStemNW', '2Suff PseudoStemNW', '2Suff w/ Lat.', '2Suff w/o Lat.', '2Suff Composite'],  {'2Suff': 102},  True)  
    
    meg_full_data.append(epochs.get_data())
    labels_full_data.append(epochs.events[:, 2])
    
    del epochs
    
meg_full_data = np.vstack(meg_full_data)
labels_full_data = np.vstack(labels_full_data)
np.save('./meg_full_data_12.npy', meg_full_data)
np.save('./labels_12.npy', labels_full_data.reshape(-1))
print("meg full data shape: ", meg_full_data.shape, labels_full_data.shape)

  0%|          | 0/24 [00:00<?, ?it/s]

/scratch/alr664/multiple_affix/meg/A0394/A0394_rejection-epo.fif


/state/partition1/job-47276493/ipykernel_1448800/661870079.py:15: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.592 s)
  epochs = epochs.crop(tmin=0. , tmax= 0.6)
/state/partition1/job-47276493/ipykernel_1448800/661870079.py:22: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  meg_full_data.append(epochs.get_data())
  4%|▍         | 1/24 [00:35<13:45, 35.89s/it]

/scratch/alr664/multiple_affix/meg/A0421/A0421_rejection-epo.fif


/state/partition1/job-47276493/ipykernel_1448800/661870079.py:15: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.592 s)
  epochs = epochs.crop(tmin=0. , tmax= 0.6)
/state/partition1/job-47276493/ipykernel_1448800/661870079.py:22: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  meg_full_data.append(epochs.get_data())
  8%|▊         | 2/24 [01:11<13:05, 35.72s/it]

...

/scratch/alr664/multiple_affix/meg/A0524/A0524_rejection-epo.fif


/state/partition1/job-47276493/ipykernel_1448800/661870079.py:15: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.592 s)
  epochs = epochs.crop(tmin=0. , tmax= 0.6)
/state/partition1/job-47276493/ipykernel_1448800/661870079.py:22: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  meg_full_data.append(epochs.get_data())
 96%|█████████▌| 23/24 [13:37<00:35, 35.52s/it]

/scratch/alr664/multiple_affix/meg/A0525/A0525_rejection-epo.fif


/state/partition1/job-47276493/ipykernel_1448800/661870079.py:15: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (0.592 s)
  epochs = epochs.crop(tmin=0. , tmax= 0.6)
/state/partition1/job-47276493/ipykernel_1448800/661870079.py:22: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  meg_full_data.append(epochs.get_data())
100%|██████████| 24/24 [14:13<00:00, 35.55s/it]


meg full data shape:  (45264, 207, 75) (24, 1886)


In [23]:
meg_full_data = np.load('./meg_full_data_12.npy')
labels_full_data = np.load('./labels_12.npy')

print(meg_full_data.shape, labels_full_data.shape)

(45264, 207, 75) (45264,)


In [24]:
## 3-class classification code
#y_cat = np.array([int(yi[0]) for yi in y])
#print(y_cat.shape)

np.unique(labels_full_data)
print(meg_full_data.shape[1:])

#labels_full_data = np.where(labels_full_data == 12, 0, labels)
#labels_full_data = np.where(labels_full_data == 14, 3, labels) 
#labels_full_data = np.where(labels_full_data == 15, 5, labels)
#labels_full_data = np.where(labels_full_data == 21, 6, labels)
#labels_full_data = np.where(labels_full_data == 22, 7, labels)
#labels_full_data = np.where(labels_full_data == 23, 8, labels)
#labels_full_data = np.where(labels_full_data == 24, 9, labels)
#labels_full_data = np.where(labels_full_data == 25, 10, labels)

(207, 75)


In [25]:
labels_full_data = np.where(labels_full_data == 12, 0, labels_full_data)
labels_full_data = np.where(labels_full_data == 14, 3, labels_full_data) 
labels_full_data = np.where(labels_full_data == 15, 5, labels_full_data)
labels_full_data = np.where(labels_full_data == 21, 6, labels_full_data)
labels_full_data = np.where(labels_full_data == 22, 7, labels_full_data)
labels_full_data = np.where(labels_full_data == 23, 8, labels_full_data)
labels_full_data = np.where(labels_full_data == 24, 9, labels_full_data)
labels_full_data = np.where(labels_full_data == 25, 10, labels_full_data)

In [26]:
np.unique(labels_full_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)

In [60]:
## Classification using GRU

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, GRU, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define the model architecture with increased complexity
input_layer = Input(shape=meg_full_data.shape[1:])

# Increase the number of units and add more layers
x = GRU(128, return_sequences=True)(input_layer)
x = Dropout(0.2)(x)  # Add dropout for regularization
x = GRU(128, return_sequences=True)(x)
x = Dropout(0.2)(x)  # Add dropout for regularization
x = GRU(128)(x)  # Last GRU layer does not return sequences
x = Dropout(0.2)(x)  # Add dropout for regularization
 
# Increase the complexity of the model further by adding Dense layers before the output
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)  # Add dropout for regularization
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)  # Add dropout for regularization
 
#output_layer = Dense(1)(x)

output_layer = Dense(len(np.unique(labels_full_data)), activation='softmax')(x)
 
model = Model(inputs=input_layer, outputs=output_layer)
 
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(meg_full_data, labels_full_data, test_size=0.2)

In [62]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=30, verbose=True)

Epoch 1/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.1418 - loss: 2.3208
Epoch 2/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1522 - loss: 2.2915
Epoch 3/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1590 - loss: 2.2873
Epoch 4/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1595 - loss: 2.2848
Epoch 5/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1632 - loss: 2.2804
Epoch 6/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1604 - loss: 2.2837
Epoch 7/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1613 - loss: 2.2871
Epoch 8/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1612 - loss: 2.2845
Epoch 9/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1618 - loss: 2.2807
Epoch 10/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1591 - loss: 2.2826
Epoch 11/30
566/566 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.1596 - loss: 2.2793
Epoch 12/30
566/566 ━━━━━━━━━━

In [63]:
# Evaluate the model on the test set
#X_test, y_test = meg_full_data[43378:], labels_full_data[43378:]
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')

283/283 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.1499 - loss: 2.2765
Test Accuracy: 0.1535


In [66]:
y_pred = model.predict(X_test)

283/283 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


In [67]:
print(y_pred)

[[0.13307685 0.01636612 0.12238953 ... 0.11022221 0.10515746 0.01740436]
 [0.13307685 0.01636612 0.12238953 ... 0.11022221 0.10515746 0.01740436]
 [0.13307685 0.01636612 0.12238953 ... 0.11022221 0.10515746 0.01740436]
 ...
 [0.13307685 0.01636612 0.12238953 ... 0.11022221 0.10515746 0.01740436]
 [0.13307685 0.01636612 0.12238953 ... 0.11022221 0.10515746 0.01740436]
 [0.13307685 0.01636612 0.12238953 ... 0.11022221 0.10515746 0.01740436]]
